<h1 style="color:#ff9966"> scRNA-seq and scATAC-seq Integration </h1>

<h3> Analysis of 10k peripheral blood mononuclear cells (PBMCs) using gene expression counts and euchromatin counts. </h3>

<a href="https://muon-tutorials.readthedocs.io/en/latest/single-cell-rna-atac/index.html" >Muon Documentation  </a>

<h4> Joint Data from 10X Genomics: </h4>
<a href="https://cf.10xgenomics.com/samples/cell-arc/1.0.0/pbmc_granulocyte_sorted_10k/pbmc_granulocyte_sorted_10k_web_summary.html" >From 10X Genomics Database  </a>

Estimated number of cells: <h3 style="color:#ECF87F"> 11,909 </h3>


In [ ]:
import anndata as ad
import pandas as pd
import numpy as np
import muon as mu
import scanpy as sc

from muon import atac as ac
import os


/home/gugu/mambaforge/envs/bioinfo/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data_dir = "data/pbmc10k"
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)


In [ ]:
# Remove file prefixes if any
prefix = "pbmc_granulocyte_sorted_10k_"
for file in os.listdir(data_dir):
    if file.startswith(prefix):
        new_filename = file[len(prefix):]
        os.rename(os.path.join(data_dir, file), os.path.join(data_dir, new_filename))

In [ ]:

mdata = mu.read_10x_h5(os.path.join(data_dir, "filtered_feature_bc_matrix.h5"))  

#Not 

mdata.var_names_make_unique()
mdata


<h2 style="color:#ff9966"> 1.1 scRNAseq Analysis</h2>


In [ ]:
rna = mdata.mod['rna']
rna

In [ ]:
rna.var['mt'] = rna.var_names.str.startswith('MT-')
print('Mitochondrial genes: \n ')

print(rna.var[rna.var['mt'] == True])
#List of mitochondrial genes. 

sc.pp.calculate_qc_metrics(rna, qc_vars=['mt'], percent_top=None, inplace=True)
sc.pl.violin(rna,  ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

In [ ]:
mu.pp.filter_var(rna, 'n_cells_by_counts', lambda x: x >= 3)

<h3>Quality Control of RNA</h3>

In [ ]:
mu.pp.filter_obs(rna, 'n_genes_by_counts', lambda x: (x >= 200) & (x < 5000))

mu.pp.filter_obs(rna, 'total_counts', lambda x: x < 15000)
mu.pp.filter_obs(rna, 'pct_counts_mt', lambda x : x < 20)

In [ ]:
sc.pl.violin(rna,  ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

<h3> Normalisation </h3>
Fit mRNA counts to a Counts Per Million normalization. Still includes mtRNA because we haven't filtered that out.
<hr style="color:#ff9966">


In [ ]:
sc.pp.normalize_total(rna, target_sum=10000) #CPM normilization, 1e4

sc.pp.log1p(rna)

<h3> Feature Selection </h3>
Select/Annotate HVGs
<hr style="color:#ff9966">


In [ ]:
sc.pp.highly_variable_genes(rna, min_mean=0.02, max_mean=4, min_disp=0.5)
#Annotates HVGs, genes that are correlated with high cell-to-cell variation in terms of RNA.
#Variance needs to be stabilized prior to selection of genes. Since scRNA has strong mean-variance relationship
#
sc.pl.highly_variable_genes(rna)

In [ ]:
np.sum(rna.var.highly_variable)

<h3 style="color:#ff9966">Scaling</h3>
<hr style="color:#ff9966">



In [ ]:
rna.raw = rna 

#Scale the log-normalised counts in a .raw slot

#Scale the log-normd RNA counts to have a 0 mean and 0 unit variance
sc.pp.scale(rna, max_value=10)

<h2 style="color:#ff9966">Analysis</h2>
<hr style="color:#ff9966">

Having filtered low-quality cells, normalised the counts matrix, and performed feature selection, we can already use this data for multimodal integration (ATACseq).
However it is usually a good idea to study individual modalities as well. Below we run:
-  PCA on the scaled matrix,
-  Compute cell neighbourhood graph
-  Perform clustering to define cell types.


<h3 style="color:#ff9966"> - PCA and Neighborhood Graph</h3>

We expect to see T cells and NK cells (CD2), B cells (CD79A), and KLF4 (monocytes).
So we'll use those markers in the PCA.

<img style="background:white" src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Hematopoiesis_simple.svg/1280px-Hematopoiesis_simple.svg.png"  height=600 />

In [ ]:
sc.tl.pca(rna, svd_solver='arpack')

In [ ]:
sc.pl.pca(rna, color=['CD2', 'CD79A', 'KLF4', 'IRF8'])

In [ ]:
sc.pl.pca_variance_ratio(rna, log=True)

In [ ]:
sc.pp.neighbors(rna, n_neighbors=10, n_pcs=20)

In [ ]:
sc.tl.leiden(rna, resolution=.5)
sc.tl.umap(rna, spread=1., min_dist=.5, random_state=11)
sc.pl.umap(rna, color="leiden", legend_loc="on data")

<h3 style="color:#ff9966"> - Marker Genes and Celltypes</h3>

In [ ]:
sc.tl.rank_genes_groups(rna, 'leiden', method='t-test')

In [ ]:
result = rna.uns['rank_genes_groups']

groups = result['names'].dtype.names

pd.set_option('display.max_columns', 50)
pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names', 'pvals']}).head(10)

In [ ]:
sc.pl.rank_genes_groups(rna, n_genes=20, sharey=False)

In [ ]:
mu.pp.filter_obs(rna, "leiden", lambda x: ~x.isin(["9", "15", "12", "16"]))

In [ ]:
new_cluster_names = {
"0": "CD4+ memory T", "1": "CD8+ naïve T", "3": "CD4+ naïve T",
"5": "CD8+ activated T", "7": "NK", "13": "MAIT",
"6": "memory B", "10": "naïve B",
"4": "CD14 mono", "2": "intermediate mono", "8": "CD16 mono",
"11": "mDC", "14": "pDC",
}

rna.obs['celltype'] = rna.obs.leiden.astype("str").values

In [ ]:
rna.obs.celltype = rna.obs.celltype.astype("category")
rna.obs.celltype = rna.obs.celltype.cat.rename_categories(new_cluster_names)

In [ ]:
rna.obs.celltype.cat.reorder_categories([
'CD4+ naïve T', 'CD4+ memory T', 'MAIT',
'CD8+ naïve T', 'CD8+ activated T', 'NK',
'naïve B', 'memory B',
'CD14 mono', 'intermediate mono', 'CD16 mono',
'mDC', 'pDC'], inplace=True)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
cmap = plt.get_cmap('rainbow')
colors = cmap(np.linspace(0, 1, len(rna.obs.celltype.cat.categories)))
rna.uns["celltype_colors"] = list(map(matplotlib.colors.to_hex, colors))

In [ ]:
sc.pl.umap(rna, color="celltype", legend_loc="on data")

In [ ]:
marker_genes = ['IL7R', 'TRAC',
'ITGB1', # CD29
'SLC4A10',
'CD8A', 'CD8B', 'CCL5',
'GNLY', 'NKG7',
'CD79A', 'MS4A1', 'IGHM', 'IGHD',
'IL4R', 'TCL1A',
'KLF4', 'LYZ', 'S100A8', 'ITGAM', # CD11b
'CD14', 'FCGR3A', 'MS4A7',
'CST3', 'CLEC10A', 'IRF8', 'TCF4']

In [ ]:

mdata.write("data/pbmc10k.h5mu")
#Read until page 49

<h2 style="color:#ff9966">1.2 Chromatin Accessability Processing</h2>
<hr style="color:#ff9966">

Here we'll use our 10K pbmc scATACseq dataset and perform similar processing to the scRNA-seq one.  Then, we can look into transferring labels and 


In [ ]:
mdata = mu.read("data/pbmc10k.h5mu")
mdata
atac = mdata.mod['atac']
atac
#atac
#variables: Gene IDs, Feature Types, Genome, Interval
#Unstructured Annotations: ATAC data

<h3 style="color:#ff9966">Quality Control</h3>

Note: A gene in the context of ATAC here should be thought of as a peak in chromatin accessability.
So the matrix is essentially a matrix of cells x peak counts


In [ ]:
sc.pp.calculate_qc_metrics(atac, inplace=True, log1p=False, percent_top=None)

In [ ]:
sc.pl.violin(atac, ['total_counts', 'n_genes_by_counts'], jitter=0.4, size=1.5, multi_panel=True)

In [ ]:
mu.pp.filter_var(atac, 'n_cells_by_counts', lambda x: x >= 10)  

In [ ]:
mu.pp.filter_obs(atac, 'n_genes_by_counts', lambda x: (x >= 2000) & (x <= 15000))
# This is analogous to
# sc.pp.filter_cells(atac, max_genes=15000)
# sc.pp.filter_cells(atac, min_genes=2000)
# but does in-place filtering avoiding copying the object
mu.pp.filter_obs(atac, 'total_counts', lambda x: (x >= 4000) & (x <= 40000))

In [ ]:
sc.pl.violin(atac, ['n_genes_by_counts', 'total_counts'], jitter=0.4, multi_panel=True, size=0.6)

In [ ]:
mu.pl.histogram(atac, ['n_genes_by_counts', 'total_counts'])

In [ ]:
atac.obs['NS'] = 1 
ac.pl.fragment_histogram(atac, region='chr1:1-2000000')

<h2 style="color:#ff9966">1.3 Integrating GEX and Chromatin Accessability</h2>
<hr style="color:#ff9966">

Here we integrate our two omics (From 1.1 and 1.2) using multi-omic factor analysis (MOFA). This requires training a MOFA model.

In [ ]:
mu.pp.intersect_obs(mdata)


In [ ]:
mdata.shape

In [ ]:
from sklearn.metrics import adjusted_rand_score as ari
ari(mdata.obs['rna:celltype'], mdata.obs['rna:celltype'])